#### COURSERA CAPSTONE PROJECT 1

To render maps in notebook open the file in nbviewer. Here is how:<br>
Go to : https://nbviewer.jupyter.org/ <br>
Type : Nikhil092<br>
Click : Coursera_Capstone<br> 
Click : Coursera_Project_Final_Notebook_1.ipynb

In [1]:
import pandas as pd
import numpy as np

#for scraping data from wikipedia
import requests
import urllib.request
from urllib.request import urlopen
from bs4 import BeautifulSoup 

#these three did not work for me for some reason. 'NoneType' error for some neighborhoods and most of the postal codes
from geopy.geocoders import Nominatim      
from geopy.geocoders import OpenMapQuest
from geopy.geocoders import GoogleV3 

# !pip install pgeocode
import pgeocode 

# map rendering library
# !pip install folium
import folium 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans

### 1.0 Data Scrapping from Wikipedia
<font size="3">I am using BeautifulSoup module for scrapping data from html file sourced from wikipedia
    </font>

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' #wikipedia link
html = urlopen(url)

In [3]:
soup = BeautifulSoup(html, 'html.parser')

In [78]:
tables = soup.find_all('table')
p=[tables[0]]

In [5]:
#Create array to hold the data we extract

postal_code = []
borough = []
neighborhood = []
i=0
j=0

for table in p:
    rows = table.find_all('tr')
    i=i+1
    
    for row in rows:
        cells = row.find_all('td')
        j=j+1
        
        if len(cells) > 1:
            rank = cells[0]
            postal_code.append(rank.text.strip())
            
            country = cells[1]
            borough.append(country.text.strip())
            
            rate = cells[2]
            neighborhood.append(rate.text.strip())

In [6]:
print( len(neighborhood) , len(borough), len(postal_code)) 

180 180 180


In [7]:
df1 = pd.DataFrame(postal_code, columns = ['postal_code'])
df1['borough'] = borough
df1['neighborhood'] = neighborhood


In [8]:
tor_neigh = df1[df1['borough'] != 'Not assigned'].reset_index(drop=True)

**Shape of the postal code DataFrame**

In [9]:
tor_neigh.shape 

(103, 3)

In [10]:
(tor_neigh['postal_code'].value_counts()>1).sum() # no postal code is repeated twice

0

In [11]:
tor_neigh.head()

,postal_code,borough,neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [12]:
print('"Not assigned" Borough - {}, "Not assigned" Neighborhood - {}'
      .format((tor_neigh['borough'] == 'Not assigned').sum(),(tor_neigh['neighborhood'] == 'Not assigned').sum()))

"Not assigned" Borough - 0, "Not assigned" Neighborhood - 0


### 2.0 Mapping Postal Codes to lat-lng

<font size="3">  To map lat-lng to the neighborhoods I tried different geopy modules including the one given in the project instructions. None of them seem to be doing the job for me.<br />
* geopy.geocoders.Nominatim worked for some neighborhood, not all, especially where there are multiple neighborhoods in the same borough. <br />
* geopy.geocoders.OpenMapQuest had the same issue. (You will have to get the api key by registering on their website)<br />
* geopy.geocoders.GoogleV3 had error with API key. <br />
</font>

<font size="3">Double click __here__ to check out the differences in values obtained vs those given in the lab instructions

<!--
#Plot this folium map to check out the differences in values obtained vs those given in the lab instructions
l1l = [43.756303 ,43.7600778, 43.7069487202267,43.7047983,43.697952,43.6628435] 
l2l = [-79.565963, -79.5717598, -79.3702648489458,-79.3680904,-79.387383,-79.3863114]

test_map = folium.Map(location=[43.756303, -79.565963], zoom_start=11)
for l1,l2 in zip(l1l,l2l):
    folium.CircleMarker(
            [l1, l2],
            radius=5,
            popup=(l1,l2),
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(test_map) 

test_map
-->

<!--
#This one gives an error

address = 'Central Bay Street,Downtown,Toronto'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)
print(location.raw)
-->

<!--
#This one worked, but did not work for boroughs with multiple neighbourhoods

address = 'Central Bay Street,Downtown,Toronto'

geolocator = OpenMapQuest(api_key='Dvx0YapTsVpJitGUgBQNqDYR0KTfFPqN')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)
print(location.raw)
-->

</font>

In [13]:
nomi = pgeocode.Nominatim('ca') #pgeocode has minimum errors

In [14]:
data_lat = []
data_long = []

for post_code in tor_neigh['postal_code']:
    ooh = nomi.query_postal_code(post_code)
    data_lat.append(round(ooh[9],4))
    data_long .append(round(ooh[10],4))

In [15]:
tor_neigh['latitude'] = data_lat
tor_neigh['longitude'] = data_long

In [16]:
tor_neigh.head()

,postal_code,borough,neighborhood,latitude,longitude
0,M3A,North York,Parkwoods,43.7545,-79.3300
1,M4A,North York,Victoria Village,43.7276,-79.3148
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889


In [17]:
#checking for missing values
tor_neigh[tor_neigh['longitude'].isnull()]

,postal_code,borough,neighborhood,latitude,longitude
76,M7R,Mississauga,Canada Post Gateway Processing Centre,NaN,NaN


In [18]:
#replacing missing value from the file in lab instructions
cheat = pd.read_csv('http://cocl.us/Geospatial_data')

In [19]:
cheat.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [20]:
cheat[cheat['Postal Code']=='M7R']

,Postal Code,Latitude,Longitude
86,M7R,43.636966,-79.615819


In [21]:
tor_neigh.loc[76,['latitude']]='43.6369'
tor_neigh.loc[76,['longitude']]='-79.6158'

In [22]:
tor_neigh[tor_neigh['longitude'].isnull()].sum()

postal_code     0.0
borough         0.0
neighborhood    0.0
latitude        0.0
longitude       0.0
dtype: float64

### 3.0 Exploring the Neighborhoods

1.0 All the boroughs

In [23]:
encoder = LabelEncoder()
encoder.fit(tor_neigh['borough'])
tor_neigh['enc_borough'] = encoder.transform(tor_neigh['borough'])

In [24]:
fixed_columns = list(tor_neigh.columns[0:2])+[tor_neigh.columns[-1]] + list(tor_neigh.columns[2:5])
tor_neigh = tor_neigh[fixed_columns]
tor_neigh.head()

,postal_code,borough,enc_borough,neighborhood,latitude,longitude
0,M3A,North York,6,Parkwoods,43.7545,-79.33
1,M4A,North York,6,Victoria Village,43.7276,-79.3148
2,M5A,Downtown Toronto,1,"Regent Park, Harbourfront",43.6555,-79.3626
3,M6A,North York,6,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504
4,M7A,Downtown Toronto,1,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889


In [25]:
location = tor_neigh.loc[0,['latitude','longitude']]

k = max(tor_neigh['enc_borough'])
x1 = np.arange(k+1)
ys1 = [i + x1 + (i*x1)**2 for i in range(k+1)]
colors_array1 = cm.rainbow(np.linspace(0, 1, len(ys1)))
rainbow1 = [colors.rgb2hex(i) for i in colors_array1]

tor_neigh_map = folium.Map(location=[location[0],location[1]], zoom_start=11)

for i,l1,l2,j in zip(tor_neigh.index,tor_neigh['latitude'],tor_neigh['longitude'],tor_neigh['enc_borough']):
    folium.CircleMarker(
            [l1, l2],
            radius=5,
            popup=(tor_neigh['borough'][i],tor_neigh['neighborhood'][i]),
            color=rainbow1[j],
            fill=True,
            fill_color=rainbow1[j],
            fill_opacity=0.7
            ).add_to(tor_neigh_map) 

tor_neigh_map

In the Map it looks like one of the neighborhood\'s Lat-lng is not correct (It was blue in color, among the orange ones when I rendered the map.) Lets use the postal codes given in lab instructions. I call it cheat sheet 

2.0 All the boroughs using the cheat sheet for lat-lng

In [26]:
tor_cheat = pd.merge(tor_neigh, cheat, left_on='postal_code', right_on='Postal Code')

In [27]:
tor_cheat.head()

,postal_code,borough,enc_borough,neighborhood,latitude,longitude,Postal Code,Latitude,Longitude
0,M3A,North York,6,Parkwoods,43.7545,-79.33,M3A,43.753259,-79.329656
1,M4A,North York,6,Victoria Village,43.7276,-79.3148,M4A,43.725882,-79.315572
2,M5A,Downtown Toronto,1,"Regent Park, Harbourfront",43.6555,-79.3626,M5A,43.654260,-79.360636
3,M6A,North York,6,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504,M6A,43.718518,-79.464763
4,M7A,Downtown Toronto,1,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889,M7A,43.662301,-79.389494


In [28]:
#dropping columns
tor_cheat.drop(columns = ['Postal Code','latitude','longitude'],axis = 1,inplace = True)
tor_cheat.head()

,postal_code,borough,enc_borough,neighborhood,Latitude,Longitude
0,M3A,North York,6,Parkwoods,43.753259,-79.329656
1,M4A,North York,6,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,1,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,6,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,1,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [29]:
location = tor_cheat.loc[0,['Latitude','Longitude']]

k = max(tor_cheat['enc_borough'])
x1 = np.arange(k+1)
ys1 = [i + x1 + (i*x1)**2 for i in range(k+1)]
colors_array1 = cm.rainbow(np.linspace(0, 1, len(ys1)))
rainbow1 = [colors.rgb2hex(i) for i in colors_array1]

tor_cheat_map = folium.Map(location=[location[0],location[1]], zoom_start=11)

for i,l1,l2,j in zip(tor_cheat.index,tor_cheat['Latitude'],tor_cheat['Longitude'],tor_cheat['enc_borough']):
    folium.CircleMarker(
            [l1, l2],
            radius=5,
            popup=(tor_cheat['borough'][i],tor_cheat['neighborhood'][i]),
            color=rainbow1[j],
            fill=True,
            fill_color=rainbow1[j],
            fill_opacity=0.7
            ).add_to(tor_cheat_map) 

tor_cheat_map

**Foursquare**<br>
Getting top 100 nearby venues in each neighborhood within 500 meter radius.

In [30]:
#Foursquare data
#put your credentials here or use the file in the code cell titled 'TOR_FILE'
LIMIT = 100
CLIENT_ID = 'UB0QPODEO0RKO2ND4DI0XKI3BUGIXQ5KCP1ZOVYHJNZMWPKJ' # your Foursquare ID 
CLIENT_SECRET = 'YRTCMOYHCDC4AIDIZI55YZKLTJ1YK21ZJA4Q1GPYVKX4KEIM' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

def getNearbyVenues(postal_code,borough, neighborhoods, latitudes, longitudes, radius=500):
    
    venues_list=[]
    i=0
    for postal_code,borough, neighborhood, lat, lng in zip(postal_code, borough, neighborhoods, latitudes, longitudes):
        i=i+1
        print(neighborhood , i)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(postal_code,
            borough,
            neighborhood, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

#     nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
#     nearby_venues.columns = ['Postal_code','Borough',
#                   'Neighborhood', 
#                   'Neighborhood Latitude', 
#                   'Neighborhood Longitude', 
#                   'Venue', 
#                   'Venue Latitude', 
#                   'Venue Longitude', 
#                   'Venue Category']
    print('GREAT JOB')
#     return(nearby_venues)
    return(venues_list)

In [32]:
TOR_venues = getNearbyVenues(postal_code = tor_cheat['postal_code'], 
                              borough=tor_cheat['borough'],
                              neighborhoods=tor_cheat['neighborhood'],
                              latitudes=tor_cheat['Latitude'],
                              longitudes=tor_cheat['Longitude'])

In [33]:
#TOR_FILE
#TOR_venues.to_csv('TOR_venues_correct.csv') 
TOR_venues = pd.read_csv('TOR_venues_correct.csv', index_col = 0) #use this file to load the foursquare data
TOR_venues.head()

,Postal_code,Borough,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M3A,North York,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,M3A,North York,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,M4A,North York,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,M4A,North York,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,M4A,North York,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [34]:
print(len(TOR_venues['Postal_code'].unique())) #checking if I got venues for all postal codes. Apparently not.
print(len(tor_cheat['postal_code'].unique()))

99
103


In [35]:
#checking if which postal code is missing
a=list(tor_cheat['postal_code'].unique())
b =list(TOR_venues['Postal_code'].unique())
temp3 = [x for x in a if x not in b]
temp3

['M9A', 'M9B', 'M2L', 'M1X']

In [36]:
#this is the information loss. I was unable to get it from Foursquare in all the ways I tried
lost=tor_cheat[tor_cheat['postal_code'].isin(['M9A', 'M9B', 'M2L', 'M1X'])]
lost

,postal_code,borough,enc_borough,neighborhood,Latitude,Longitude
5,M9A,Etobicoke,4,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
11,M9B,Etobicoke,4,"West Deane Park, Princess Gardens, Martin Grov...",43.650943,-79.554724
45,M2L,North York,6,"York Mills, Silver Hills",43.757490,-79.374714
95,M1X,Scarborough,7,Upper Rouge,43.836125,-79.205636


In [37]:
#Neighborhood Dataframe for which venues are available
TOR_neighbor = TOR_venues.groupby(['Postal_code', 'Borough', 'Neighborhood',
                          'Neighborhood Latitude', 'Neighborhood Longitude']).sum()
TOR_neighbor.reset_index(inplace=True)
TOR_neighbor.drop(columns = ['Venue Latitude','Venue Longitude'],inplace = True)
TOR_neighbor.head()

,Postal_code,Borough,Neighborhood,Neighborhood Latitude,Neighborhood Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [38]:
#Unique venue categories
len(TOR_venues['Venue Category'].unique())

269

**Downtown Toronto seems to be the most happening area of Toronto, densely occupied by hangout spots**

In [39]:
TOR_venues.groupby('Borough')['Borough'].count().sort_values()

Borough
Mississauga           13
York                  17
Etobicoke             68
East York             74
Scarborough           87
Central Toronto      112
East Toronto         122
West Toronto         157
North York           236
Downtown Toronto    1241
Name: Borough, dtype: int64

In [40]:
TOR_venues.shape 

(2127, 9)

In [41]:
TOR_dummies = pd.get_dummies(TOR_venues['Venue Category'])

In [42]:
#adding columns
TOR_dummies['Borough']=TOR_venues['Borough']
TOR_dummies['Neighborhood']=TOR_venues['Neighborhood']
TOR_dummies['Postal_code']=TOR_venues['Postal_code']

In [43]:
#correcting columns' orders
col = list(TOR_dummies.columns)
col1 = [i for i in col if i not in ['Postal_code','Neighborhood','Borough']]
col1 = ['Postal_code','Neighborhood','Borough'] + col1
TOR_dummies = TOR_dummies[col1]

In [44]:
TOR_dummies.head()

,Postal_code,Neighborhood,Borough,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M3A,Parkwoods,North York,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M3A,Parkwoods,North York,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M4A,Victoria Village,North York,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M4A,Victoria Village,North York,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M4A,Victoria Village,North York,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [45]:
#Frequency table of all the venues
p = TOR_dummies.groupby(['Postal_code','Neighborhood']).sum()
p.head()

,,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
Postal_code,Neighborhood,,,,,,,,,,,,,,,,,,,,,
M1B,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
M1C,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
M1E,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
M1G,Woburn,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
M1H,Cedarbrae,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [46]:
#frequency of venues in each neighborhood
fre = p.sum(axis=1) 
freq=pd.DataFrame(fre,columns=['frequency'])
freq.reset_index(inplace=True)
freq.head()

,Postal_code,Neighborhood,frequency
0,M1B,"Malvern, Rouge",2
1,M1C,"Rouge Hill, Port Union, Highland Creek",2
2,M1E,"Guildwood, Morningside, West Hill",8
3,M1G,Woburn,3
4,M1H,Cedarbrae,8


In [47]:
#Dataset for plotting Neighborhood with frequency of venues
TOR_neighbor_freq = pd.merge(TOR_neighbor,freq,left_on=['Neighborhood','Postal_code'],right_on=['Neighborhood','Postal_code'])
encoder.fit(TOR_neighbor_freq['Borough'])
TOR_neighbor_freq['enc_borough'] = encoder.transform(TOR_neighbor_freq['Borough'])
TOR_neighbor_freq.head()

,Postal_code,Borough,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,frequency,enc_borough
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,2,7
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,2,7
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,8,7
3,M1G,Scarborough,Woburn,43.770992,-79.216917,3,7
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,8,7


In [48]:
#the radius of the marker shows how densely the neighborhood is filled with different venues
location = TOR_neighbor_freq.loc[25,['Neighborhood Latitude','Neighborhood Longitude']]

k = max(TOR_neighbor_freq['enc_borough'])
x1 = np.arange(k+1)
ys1 = [i + x1 + (i*x1)**2 for i in range(k+1)]
colors_array1 = cm.rainbow(np.linspace(0, 1, len(ys1)))
rainbow1 = [colors.rgb2hex(i) for i in colors_array1]

TOR_neighbor_freq_map = folium.Map(location=[location[0],location[1]], zoom_start=11)

for i,l1,l2,j,f in zip(TOR_neighbor_freq.index,TOR_neighbor_freq['Neighborhood Latitude'],TOR_neighbor_freq['Neighborhood Longitude'],TOR_neighbor_freq['enc_borough']
                     ,TOR_neighbor_freq['frequency']):
    folium.CircleMarker(
            [l1, l2],
            radius=5+f/5,
            popup=(TOR_neighbor_freq['Borough'][i],TOR_neighbor_freq['Neighborhood'][i]),
            color=rainbow1[j],
            fill=True,
            fill_color=rainbow1[j],
            fill_opacity=0.7
            ).add_to(TOR_neighbor_freq_map) 

TOR_neighbor_freq_map

Making clusters based on the types of venues in the locality and then plotting them. Also encorporating the frequency of venues through bubble size

In [50]:
#making a one zero matrix. one, if the neighborhood has even one venue of that venue category in the locality. Zero otherwise.
p[p>0]
q = p>0
ven_type = q.astype(int)
ven_type.head()

,,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
Postal_code,Neighborhood,,,,,,,,,,,,,,,,,,,,,
M1B,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
M1C,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
M1E,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
M1G,Woburn,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
M1H,Cedarbrae,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [51]:
# set number of clusters
kclusters = 8

#manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ven_type)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:99] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 7, 1, 7, 1, 0, 1, 1, 7, 1,
       7, 1, 1, 7, 7, 1, 1, 1, 1, 1, 1, 1, 1, 1, 7, 7, 1, 6, 7, 6, 1, 1,
       6, 6, 1, 7, 1, 6, 6, 6, 5, 2, 4, 6, 3, 3, 3, 3, 6, 1, 1, 7, 6, 6,
       1, 4, 3, 1, 1, 1, 1, 6, 1, 6, 6, 1, 1, 1, 6, 1, 6, 6, 7, 1, 7, 7,
       1, 1, 1, 7, 1, 1, 1, 7, 1, 7, 1])

In [52]:
import copy
TOR_neighbor_cluster1 = copy.deepcopy(TOR_neighbor_freq)
TOR_neighbor_cluster1.insert(3,'Cluster_label',kmeans.labels_)

In [53]:
TOR_neighbor_cluster1.head()

,Postal_code,Borough,Neighborhood,Cluster_label,Neighborhood Latitude,Neighborhood Longitude,frequency,enc_borough
0,M1B,Scarborough,"Malvern, Rouge",1,43.806686,-79.194353,2,7
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",1,43.784535,-79.160497,2,7
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",1,43.763573,-79.188711,8,7
3,M1G,Scarborough,Woburn,1,43.770992,-79.216917,3,7
4,M1H,Scarborough,Cedarbrae,1,43.773136,-79.239476,8,7


In [54]:
#the radius of the marker shows how densely the neighborhood is filled with different venues
location = TOR_neighbor_cluster1.loc[25,['Neighborhood Latitude','Neighborhood Longitude']]

k = max(TOR_neighbor_cluster1['Cluster_label'])
x1 = np.arange(k+1)
ys1 = [i + x1 + (i*x1)**2 for i in range(k+1)]
colors_array1 = cm.rainbow(np.linspace(0, 1, len(ys1)))
rainbow1 = [colors.rgb2hex(i) for i in colors_array1]

TOR_neighbor_cluster1_map = folium.Map(location=[location[0],location[1]], zoom_start=11)

for i,l1,l2,j,f in zip(TOR_neighbor_cluster1.index,TOR_neighbor_cluster1['Neighborhood Latitude'],TOR_neighbor_cluster1['Neighborhood Longitude'],TOR_neighbor_cluster1['Cluster_label']
                     ,TOR_neighbor_cluster1['frequency']):
    folium.CircleMarker(
            [l1, l2],
            radius=5+f/5,
            popup=(TOR_neighbor_cluster1['Borough'][i],TOR_neighbor_cluster1['Neighborhood'][i]),
            color=rainbow1[j],
            fill=True,
            fill_color=rainbow1[j],
            fill_opacity=0.7
            ).add_to(TOR_neighbor_cluster1_map) 

TOR_neighbor_cluster1_map

In [55]:
TOR_neighbor_cluster1['Cluster_label'].value_counts()

1    55
7    17
6    17
3     5
4     2
5     1
2     1
0     1
Name: Cluster_label, dtype: int64

* There are 3 exclusive cluster(may vary when changing the no_cluster). No other cluster is like theirs i.e no other neighborhood has venues similar to them.
* Density of the clusters is maximum near Downtown Toronto.
* More that half the neighborhoods have similar venues.

**Analysis similar to New York lab, that is, using mean frequency instead of zero-one matrix I prepared**

In [56]:
#using mean frequency instead of zero-one matrix
pn = TOR_dummies.groupby(['Postal_code','Neighborhood']).mean()
pn.head()

,,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
Postal_code,Neighborhood,,,,,,,,,,,,,,,,,,,,,
M1B,"Malvern, Rouge",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
M1C,"Rouge Hill, Port Union, Highland Creek",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
M1E,"Guildwood, Morningside, West Hill",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
M1G,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
M1H,Cedarbrae,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [73]:
# set number of clusters
kclusters = 8

#manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(pn)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:99] 

array([4, 5, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1,
       0, 1, 0, 0, 0, 0, 1, 1, 3, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 7, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 3, 0, 0, 6, 3, 2, 0, 7, 0, 0])

In [74]:
import copy
TOR_neighbor_cluster1l = copy.deepcopy(TOR_neighbor_freq)
TOR_neighbor_cluster1l.insert(3,'Cluster_label',kmeans.labels_)
TOR_neighbor_cluster1l.head()

,Postal_code,Borough,Neighborhood,Cluster_label,Neighborhood Latitude,Neighborhood Longitude,frequency,enc_borough
0,M1B,Scarborough,"Malvern, Rouge",4,43.806686,-79.194353,2,7
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",5,43.784535,-79.160497,2,7
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",0,43.763573,-79.188711,8,7
3,M1G,Scarborough,Woburn,0,43.770992,-79.216917,3,7
4,M1H,Scarborough,Cedarbrae,0,43.773136,-79.239476,8,7


In [77]:
#the radius of the marker shows how densely the neighborhood is filled with different venues
location = TOR_neighbor_cluster1l.loc[25,['Neighborhood Latitude','Neighborhood Longitude']]

k = max(TOR_neighbor_cluster1l['Cluster_label'])
x1 = np.arange(k+1)
ys1 = [i + x1 + (i*x1)**2 for i in range(k+1)]
colors_array1 = cm.rainbow(np.linspace(0, 1, len(ys1)))
rainbow1 = [colors.rgb2hex(i) for i in colors_array1]

TOR_neighbor_cluster1l_map = folium.Map(location=[location[0],location[1]], zoom_start=11)

for i,l1,l2,j,f in zip(TOR_neighbor_cluster1l.index,TOR_neighbor_cluster1l['Neighborhood Latitude'],TOR_neighbor_cluster1l['Neighborhood Longitude'],TOR_neighbor_cluster1l['Cluster_label']
                     ,TOR_neighbor_cluster1l['frequency']):
    folium.CircleMarker(
            [l1, l2],
            radius=5,#+f/5
            popup=(TOR_neighbor_cluster1l['Borough'][i],TOR_neighbor_cluster1l['Neighborhood'][i]),
            color=rainbow1[j],
            fill=True,
            fill_color=rainbow1[j],
            fill_opacity=0.7
            ).add_to(TOR_neighbor_cluster1l_map) 

TOR_neighbor_cluster1l_map

In [76]:
TOR_neighbor_cluster1l['Cluster_label'].value_counts()

0    78
1    10
3     3
7     2
5     2
2     2
6     1
4     1
Name: Cluster_label, dtype: int64

* I am not sure about the differences in the maps in both the cases. If you find the reason why one is an accurate measure over another, I would appreciate if you  could add a comment and commit it to my git repository or you can leave a comment in coursera. Thanks a lot.